In [72]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

r = requests.get('https://www.worldometers.info/coronavirus/')
soup = BeautifulSoup(r.text, 'lxml')


In [75]:
df = pd.read_html('https://www.worldometers.info/coronavirus/')[0]

In [76]:
df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
0,China,80824,+11,3189.0,13.0,65573.0,12062,3610.0,56.2
1,Italy,21157,"+3,497",1441.0,175.0,1966.0,17750,1518.0,349.9
2,Iran,12729,"+1,365",611.0,97.0,4339.0,7779,NaN,151.5
3,S. Korea,8086,+107,72.0,5.0,714.0,7300,59.0,157.7
4,Spain,6313,"+1,081",193.0,60.0,517.0,5603,293.0,135.0
...,...,...,...,...,...,...,...,...,...
146,Suriname,1,NaN,NaN,NaN,NaN,1,NaN,NaN
147,Eswatini,1,+1,NaN,NaN,NaN,1,NaN,0.9
148,Togo,1,NaN,NaN,NaN,NaN,1,NaN,0.1
149,U.S. Virgin Islands,1,+1,NaN,NaN,NaN,1,NaN,NaN


In [32]:
# import geocoder
# lat = []
# lng = []
# for i in range(len(pickled)):
#     g = geocoder.arcgis(pickled['Country,Other'].iloc[i])
#     lat.append(g.json['lat'])
#     lng.append(g.json['lng'])

# pickled['lat'] = lat
# pickled['lng'] = lng

In [94]:
import geocoder
g = geocoder.arcgis('Spain')
g.json

{'address': 'Spain',
 'bbox': {'northeast': [49.88794966000003, 6.013936895000061],
  'southwest': [30.567949660000036, -13.30606310499994]},
 'confidence': 1,
 'lat': 40.227949660000036,
 'lng': -3.6460631049999392,
 'ok': True,
 'quality': 'Locality',
 'raw': {'name': 'Spain',
  'extent': {'xmin': -13.30606310499994,
   'ymin': 30.567949660000036,
   'xmax': 6.013936895000061,
   'ymax': 49.88794966000003},
  'feature': {'geometry': {'x': -3.6460631049999392, 'y': 40.227949660000036},
   'attributes': {'Score': 100, 'Addr_Type': 'Locality'}}},
 'score': 100,
 'status': 'OK'}

In [77]:
pickled = pd.read_pickle("./coronavirus.pkl")
df['lat'] = pickled['lat']
df['lng'] = pickled['lng'] 

In [80]:
df['ratio'] = round(df['TotalDeaths']/df['TotalCases']*100, 2)
df[['Country,Other', 'ratio', ]].dropna().head()

 

,"Country,Other",ratio
0,China,3.95
1,Italy,6.81
2,Iran,4.80
3,S. Korea,0.89
4,Spain,3.06


In [81]:
from math import ceil
def get_radius(rat):
    if rat > 10:
        return 10
    elif rat < 1:
        return 1
    else:
        return ceil(rat)

In [83]:
dropped = df[['Country,Other', 'ratio', 'lat', 'lng', 'TotalCases']].dropna()
dropped['percent'] = round(dropped['TotalCases']/dropped['TotalCases'].sum(), 2)*100
dropped['percent'].head()

0    26.0
1     7.0
2     4.0
3     3.0
4     2.0
Name: percent, dtype: float64

In [91]:
import folium
fol_map = folium.Map(location=[36.567348, 103.930027], zoom_start=2)
for lat, lon, percent, total, country in zip(
                            dropped['lat'].tolist(), 
                            dropped['lng'].tolist(), 
                            dropped['percent'].tolist(),
                            dropped['TotalCases'].tolist(),
                            dropped['Country,Other'].tolist()
                                 ):
    folium.CircleMarker(location=[lat, lon], radius = get_radius(percent), popup= str(country)+' ' + str(total)+" total cases").add_to(fol_map)

In [92]:
fol_map